# 2. Build a Recommendation System for Purchase Data

The scope of this notebook is 

- Prepare the Scoring Function
- Unit Test the Score
- Build the Dash Application

The Business case is an mobile app allowing its customers to place orders before they even have to walk into the store.
When a customer first taps on the “order” page, we may recommend 

- Personalized recommendation with ranked list of items (product IDs) that the user is most likely to want to put in his/her (empty) “basket”

Assuming that the scenario is ModelOps 0. Then: 

1. Data scientists hand over a trained model as an artifact to the engineering team for deployement
2. The handoff can include putting the trained model in the models registry
3. The Scoring process is in Batch on a sigle EC2 instance

We have to reproduce the required development enviroment

0. Define Artefacter function to get the last version of Champion Model (optional)

1. Define Scoring Functions: Batch scoring is the main assumption

    - Define the get_top_items function 
    - Define the get_top_n_ui function
    

2. Unit Test 

3. Define a quick front end that simulate Mobile App (Test it in Docker)


## Settings

### Import libraries

In [1]:
#Data
import sqlalchemy as sql

#Data Science
import pandas as pd
from surprise import dump

#Model Tracking
import mlflow
from mlflow.tracking import MlflowClient

#Utils
import os
import glob
import logging
from collections import defaultdict
import configparser
import json
import pickle
import unittest
import docker
import request

#Settings
from pprint import pprint
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Set enviroment variables

In [2]:
# Enviroment variables
outmodels = '../models/'
app_folder = '../src/app'

# Set dbconnection variables
dbconnPath = './dbconn.properties'
config = configparser.RawConfigParser()
config.read(dbconnPath)
params = config
db_host=params.get('CONN', 'host')
db_port=params.get('CONN', 'port')
db_user=params.get('CONN', 'user')
db_pwd=params.get('CONN', 'password')
db_name=params.get('CONN', 'database')

# Set connection string
connection_str = f'mysql+pymysql://{db_user}:{db_pwd}@{db_host}:{db_port}/{db_name}'

### Download Model Artefact from Mlflow server

In [3]:
client = MlflowClient()
for regmodel in client.list_registered_models():
    regmodel_info = dict(regmodel)

# pprint(regmodel_info, indent=3)

champion=client.get_registered_model('Champion')
championid=champion.latest_versions[-1].run_id

art_list = [arts.path for arts in client.list_artifacts(championid, path=None)]

for art_path in art_list: 
    client.download_artifacts(championid, art_path, outmodels)

/opt/conda/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, MutableMapping


### Analyze the Model Artifact

In [4]:
modelpkl = [modelpath for modelpath in glob.glob(outmodels + 'model/*.pkl')][0]
modelpkl

predictions, algo = dump.load(modelpkl)

print('\n')
print('Sample of Predictions: ')
print('\n', predictions[0:10])
print('\n', 'Number of predictions:', len(predictions))



Sample of Predictions: 

 [Prediction(uid='100', iid='0', r_ui=1.0, est=1.4104866760238497, details={'was_impossible': False}), Prediction(uid='100', iid='118', r_ui=2.0, est=1.6290418315812665, details={'was_impossible': False}), Prediction(uid='100', iid='201', r_ui=1.0, est=1.0281647548199493, details={'was_impossible': False}), Prediction(uid='100', iid='24', r_ui=2.0, est=1.065194864590783, details={'was_impossible': False}), Prediction(uid='100', iid='27', r_ui=4.0, est=1.4686861264367166, details={'was_impossible': False}), Prediction(uid='100', iid='282', r_ui=6.0, est=1.2631693065495946, details={'was_impossible': False}), Prediction(uid='100', iid='51', r_ui=0.0, est=1.3060476975670121, details={'was_impossible': False}), Prediction(uid='100', iid='6', r_ui=0.0, est=1.8579018429484466, details={'was_impossible': False}), Prediction(uid='100', iid='62', r_ui=2.0, est=1.0656423124377505, details={'was_impossible': False}), Prediction(uid='100', iid='67', r_ui=3.0, est=1.55989

## Machine Learning Engineering

### Scoring Function

We have to return Top 10 Recommended Items by userid

#### Define the scoring function

In [5]:
def get_top(predictions, n=10):
    
    '''
    Returns the the top-N recommendation from a set of predictions
    
    '''
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
        
    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        
    return top_n

def get_top_n_ui(top, uid):
    try:
        top_n_ui = [[iid for (iid, _) in user_ratings] for UID, user_ratings in top.items() if UID==uid][0]
        return top_n_ui
    except ValueError: # user was not part of the trainset
        return 0

#### Unit test

In [6]:
class TestScoreFunction(unittest.TestCase):
    
    def setUp(self):
        self.testcase = "100"
        self.expected = ['6', '118', '67', '27', '0', '51', '282', '62', '24', '201']
    
    def test_empty(self):
        self.assertTrue(bool(get_top_n_ui(get_top(predictions), self.testcase)))

    def test_basic(self):
        self.assertEqual(get_top_n_ui(get_top(predictions), self.testcase), self.expected)
        
unittest.main(argv = ['first-arg-is-ignored'], exit = False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.288s

OK


### Create the Scoring Flask App

Because we're testing, I need a app folder with:

1. app.py
2. requirements.txt
3. Dockerfile

Then run the application with Docker Client and test it 

#### Create a app folder

In [7]:
if not os.path.exists(app_folder):
    os.makedirs(app_folder)
os.chdir(app_folder)

#### Write the app.py

In [8]:
%%writefile app.py

from flask import Flask

#create an instance
app = Flask(__name__)

#what URL should trigger our function
@app.route('/')
def hello_world():
    return 'Hello, World!'
            
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=9999, debug=True)

Overwriting app.py


#### Write the requirements.txt

In [9]:
%%writefile requirements.txt

Flask==0.10.1

Overwriting requirements.txt


#### Write the dockerfile

In [10]:
%%writefile Dockerfile

FROM ubuntu:16.04

RUN apt-get update -y && \
    apt-get install -y python-pip python-dev

COPY ./requirements.txt /app/requirements.txt

WORKDIR /app

RUN pip install -r requirements.txt

COPY . /app

ENV FLASK_APP=app.py

ENTRYPOINT [ "python" ]

CMD [ "app.py" ]

Overwriting Dockerfile


#### Run the Application

In [12]:
dockercli = docker.DockerClient()
dockercli.images.build(path='.', tag = 'score-flask_app:1')

(<Image: 'score-flask_app:1', 'test:latest'>,
 <itertools._tee at 0x7fcea1955e60>)

In [20]:
app_container = dockercli.containers.run('score-flask_app:1', detach=True, ports={'9999/tcp': 9999})
app_container.logs()

b' * Running on http://0.0.0.0:9999/ (Press CTRL+C to quit)\n * Restarting with stat\n * Debugger is active!\n * Debugger PIN: 729-494-560\n'

#### Test the app